In [31]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random


In [2]:
#read all the data:
items = pd.read_csv("dressipi_recsys2022/item_features.csv")
candidate = pd.read_csv("dressipi_recsys2022/candidate_items.csv")
purchase =  pd.read_csv("dressipi_recsys2022/train_purchases.csv")
sessions =  pd.read_csv("dressipi_recsys2022/train_sessions.csv")
sessions_test = pd.read_csv("dressipi_recsys2022/test_final_sessions.csv")
leaderboard = pd.read_csv("dressipi_recsys2022/test_leaderboard_sessions.csv")

In [7]:
#set the date column to a date format
purchase['date'] = pd.to_datetime(purchase['date'])
#list of unique items:
items_unique = items.item_id.unique()

In [4]:
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]

In [5]:
sessions_item_dict = {}
item_sessions_dict = {}
for i,x in enumerate(zip(sessions['session_id'], sessions['item_id'])):
    session, item = x
    if session not in sessions_item_dict:
        sessions_item_dict[session] = {}
    
    if item not in item_sessions_dict:
        item_sessions_dict[item] = {}
        
    if item not in sessions_item_dict[session]:
        sessions_item_dict[session][item] = 0
    
    if session not in item_sessions_dict[item]:
        item_sessions_dict[item][session] = 0
        
    sessions_item_dict[session][item] += 1
    item_sessions_dict[item][session] += 1

In [6]:
def generate_positive(max_num = 25):
    '''
    Generates Samples of Products seen in the same Session
    with a maximum amount of max_num items per item
    '''
    positive = []
    positive_dict = {}
    for item in tqdm(item_sessions_dict):
        for session in item_sessions_dict[item]:
            cont = 0
            for elem in sessions_item_dict[session]:
                if(elem != item):
                    positive.append([item,elem])
                    cont += 1
                    if item not in positive_dict:
                        positive_dict[item] = []
                    positive_dict[item].append(elem)
                
                if(cont == 50): break
                
    return positive, positive_dict

positive_sample,positive_dict = generate_positive() #generates data of positive samples    

100%|██████████| 23496/23496 [00:31<00:00, 756.27it/s] 


In [32]:
def generate_negative(max_num = 100):
    '''Generates Samples of Products not seen in the same Session
    with a maximum amount of max_num items per item'''
    negative = []
    for item in items_unique:
        for i in range(max_num):
            negative_sample = random.choice(items_unique)
            if negative_sample not in positive_dict[item] and negative_sample != item:
                negative.append([item,negative_sample])
    return negative   

In [35]:
negative_sample = generate_negative()

KeyError: 7

In [39]:
positive_dict[7]

KeyError: 7

In [41]:
items[items.item_id == 7]

,item_id,feature_category_id,feature_value_id
61,7,56,153
62,7,50,708
63,7,63,816
64,7,41,759
65,7,26,268
66,7,32,902
67,7,61,706
68,7,68,739
69,7,69,592
70,7,7,536


In [52]:
sessions

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211
...,...,...,...
4743815,4440001,20409,2020-10-30 23:37:20.658
4743816,4440001,14155,2020-10-30 23:31:56.607
4743817,4440001,14303,2020-10-30 23:36:17.934
4743818,4440001,27852,2020-10-30 23:39:55.186
